# AI on a GPU


In this notebook, we'll create our biggest neural networks so far and compare their training speeds between GPU and CPU training.

A lot of this notebook mirrors the PyTorch's [guide](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html) on GPU training.

## What is a GPU?

A GPU is a type of processor designed for performing many operations in parallel. GPU stands for Graphics Processing Unit. 

Contrast this to a CPU. CPU stands for Central Processing Unit. CPUs can usually perform faster, but can only process one thing at a time.


## Do I have a GPU?

Not all computers have a GPU. The code below shows you how to check if you have a GPU that `torch` can use.

This package adds support for CUDA tensor types, that implement the same function as CPU tensors, but they utilize GPUs for computation.

In [2]:
import torch
# import pyprofiler

cuda_available = torch.cuda.is_available() # check if cuda is available

print('Can i continue with this notebook?', cuda_available)

Can i continue with this notebook? True


## Firstly let's create a CNN

In [3]:
import torchvision
import torchvision.transforms as transforms
from time import time
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

class CNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.pool = torch.nn.MaxPool2d(2, 2)
        self.conv = torch.nn.Sequential(
            torch.nn.Conv2d(3, 6, 5),
            torch.nn.ReLU(),
            self.pool,
            torch.nn.Conv2d(6, 16, 5),
            torch.nn.ReLU(),
            self.pool
        )
        self.fc = torch.nn.Sequential(
            torch.nn.Linear(16 * 5 * 5, 120),
            torch.nn.ReLU(),
            torch.nn.Linear(120, 84),
            torch.nn.ReLU(),
            torch.nn.Linear(84, 10)
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(-1, 16 * 5 * 5)
        x = self.fc(x)
        return x


/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Files already downloaded and verified


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ice/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3319, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-6715e48ce668>", line 13, in <module>
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)
NameError: name 'torch' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ice/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2034, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ice/.local/lib/python3.6/site-packages/IPython/core/ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_c

NameError: name 'torch' is not defined

Let's make a function to train a model on our CPU.

In [ ]:
def trainCPU(model, dataloader, criterion, optimiser, writer, epochs=1):
    start = time()
    for epoch in range(epochs):
        for idx, (x, y) in enumerate(dataloader):
            h = model(x)
            loss = criterion(h, y)
            loss.backward()
            optimiser.step()
            optimiser.zero_grad()
            writer.add_scalar('Train/Loss', loss, epoch*len(dataloader) + idx)
        print(f'Epoch: {epoch}\tBatch: {idx}\tLoss: {loss.data}')
    duration = time() - start
    print('Time taken on CPU:', duration)

nn = CNN()          
criterion = torch.nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(nn.parameters())

trainCPU(nn, train_loader, criterion, optimiser, writer)


Now let's train the same model for one epoch on a GPU. PyTorch makes this very easy - and not a lot changes at all.

In [2]:
def trainGPU(model, dataloader, criterion, optimiser, writer, epochs=1):
    model.to(device) # THIS LINE IS NEW - it recursively move all of the models parameters to the device
    for epoch in range(epochs):
        for idx, (x, y) in enumerate(dataloader):
            x, y = x.to(device), y.to(device) # THIS LINE IS NEW - move the example features and labels to the GPU 
            h = model(x)
            loss = criterion(h, y)
            loss.backward()
            optimiser.step()
            optimiser.zero_grad()
            writer.add_scalar('Train/Loss', loss, epoch*len(dataloader) + idx)
        print(f'Epoch: {epoch}\tBatch: {idx}\tLoss: {loss.data}')
    duration = time() - start
    print('Time taken on GPU:', duration)
        
device = torch.device('cuda' if cuda_available else 'cpu')
print('DEVICE:', device)
                  

nn = CNN()
nn.to(device)
optimiser = torch.optim.Adam(nn.parameters())
trainGPU(nn, train_loader, criterion, optimiser, writer, gpu=True)


NameError: name 'torch' is not defined